## Criação arquivos lookup ##

In [1]:
import json
import pandas as pd
import os

# Carregamento dos Dados de Tráfego
print("Carregando dados de tráfego...")
df_trafego = pd.read_csv("trafego_rede_simulado.csv")

# Criar diretório se não existir
os.makedirs("../data/graph", exist_ok=True)

# Gerar lookup para IPs únicos
unique_ips = pd.concat([df_trafego['src_ip'], df_trafego['dst_ip']]).unique()
ip_lookup = {str(ip): idx for idx, ip in enumerate(unique_ips)}

# Gerar lookup para portas únicas
unique_ports = df_trafego['dst_port'].unique()
port_lookup = {str(port): idx for idx, port in enumerate(unique_ports)}

# Salvar os dicionários como JSON
with open("../data/graph/ip_lookup.json", "w") as file:
    json.dump(ip_lookup, file, indent=4)

with open("../data/graph/port_lookup.json", "w") as file:
    json.dump(port_lookup, file, indent=4)

print("Arquivos ip_lookup.json e port_lookup.json recriados com sucesso!")


Carregando dados de tráfego...
Arquivos ip_lookup.json e port_lookup.json recriados com sucesso!


## Criação do corpus e das features ##

In [2]:
import json
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import pickle
import sys
import os
sys.path.append('../')

from src.preprocessing import *
from src.preprocessing.nlp import *

# Definição de parâmetros para filtragem e análise
FILTER = 5      # Número mínimo de pacotes para considerar
TOP_PORTS = 2500  # Quantidade de portas analisadas

# Carregamento dos dados de tráfego da rede
print("Carregando dados de tráfego...")
df_trafego = pd.read_csv("trafego_rede_simulado.csv")
print(df_trafego.head())

# Aplicação do filtro de pacotes antes da renomeação de colunas
df_trafego = apply_packets_filter(df_trafego, FILTER)

# Renomeação de colunas para padronização
df_trafego.rename(columns={'dst_ip': 'destino', 'pacotes': 'pkts'}, inplace=True)

processed_df = [df_trafego]

# Carregamento do Ground Truth
print("Carregando dados de Ground Truth...")
gt = pd.read_csv("ground_truth_simulado.csv")

# Geração de estatísticas gerais
print("Calculando estatísticas gerais...")
df_merged = pd.concat(processed_df, ignore_index=True)[['src_ip', 'destino', 'pkts']]
df_merged = df_merged.merge(gt, on='src_ip', how='left').dropna()

print(df_merged.groupby('label').agg({
    'src_ip': lambda x: len(set(x)),  # Número de IPs únicos
    'destino': lambda x: len(set(x)),  # Número de destinos únicos
    'pkts': sum  # Soma total de pacotes
}))

# Carregamento da matriz de adjacência do grafo da rede
print("Carregando matriz de adjacência...")
df_grafo = pd.read_csv("matriz_adjacencia_simulada.txt", sep=" ", names=["origem", "destino", "peso"])

# Cálculo de estatísticas do grafo
total_nos = pd.concat([df_grafo['origem'], df_grafo['destino']]).nunique()
total_arestas = df_grafo.shape[0]

print(f"Total de Nós (IPs distintos): {total_nos}")
print(f"Total de Arestas (Conexões na Rede): {total_arestas}")

# Carregamento de dicionários de lookup
print("Carregando dicionários de lookup...")
with open("../data/graph/ip_lookup.json", "r") as file:
    ip_lookup = json.load(file)

with open("../data/graph/port_lookup.json", "r") as file:
    port_lookup = json.load(file)

# Extração de features dos nós com base nos dicionários de lookup
print("Extraindo features dos nós...")
df_merged['origem_id'] = df_merged['src_ip'].map(ip_lookup)
df_merged['destino_id'] = df_merged['destino'].map(ip_lookup)

node_features = df_merged.groupby('origem_id').agg({'pkts': ['mean', 'sum']}).reset_index()
node_features.columns = ['origem_id', 'media_pkts', 'soma_pkts']

print("Features extraídas com sucesso:")
print(node_features.head())

# Salvamento dos dados processados
os.makedirs("../data/features", exist_ok=True)
for day in df_trafego['interval'].unique():
    snapshot = df_trafego[df_trafego['interval'] == day]
    snapshot['origem_id'] = snapshot['src_ip'].map(ip_lookup)
    snapshot['destino_id'] = snapshot['destino'].map(ip_lookup)
    node_features_day = snapshot.groupby('origem_id').agg({'pkts': ['mean', 'sum']}).reset_index()
    node_features_day.columns = ['origem_id', 'media_pkts', 'soma_pkts']
    node_features_day.to_csv(f"../data/features/features_{day}.csv", index=False)
    print(f"Features do dia {day} salvas em '../data/features/features_{day}.csv'.")

print("Extração e salvamento de features concluídos! 🚀")

# Análise de tráfego para NLP
print("Analisando tráfego para NLP...")
raw_df = df_trafego.copy()

# Geração do corpus NLP
print("Gerando corpus NLP...")
tot_intervals = sorted(raw_df['interval'].unique())

for day in tqdm(tot_intervals):
    # Filtragem do tráfego do dia
    snapshot = raw_df[raw_df['interval'] == day].sort_values('ts')
    # Agrupamento de IPs de origem por porta de destino
    corpus = snapshot.groupby('dst_port')['src_ip'].apply(list).to_dict()
    # Ordenação das portas e alinhamento do corpus
    port_order = list(corpus.keys())
    corpus_list = [corpus[port] for port in port_order]
    # Criar diretório de saída
    os.makedirs("../data/corpus", exist_ok=True)
    # Salvamento do corpus NLP
    with open(f'../data/corpus/corpus_{day}.pkl', 'wb') as file:
        pickle.dump({'ports': port_order, 'corpus': corpus_list}, file)
    print(f"Corpus do dia {day} salvo com sucesso.")


Carregando dados de tráfego...
         src_ip        dst_ip  pacotes  bytes_transferidos protocolo  \
0   192.168.1.3  192.168.1.27      981               51284       UDP   
1  192.168.1.33  192.168.1.34      476               62016      ICMP   
2   192.168.1.6  192.168.1.27      702               74072       TCP   
3  192.168.1.50  192.168.1.17      634               62287       UDP   
4  192.168.1.10  192.168.1.43      558               13084       TCP   

   interval   ts  dst_port  
0        14  1.0     15579  
1        18  2.0     12631  
2        18  3.0     39440  
3        29  4.0      6357  
4         7  5.0     37372  
Carregando dados de Ground Truth...
Calculando estatísticas gerais...
                 src_ip  destino  pkts
label                                 
benign                1        6  2740
censys                1        6  2968
internetcensus        1        6  3473
mirai                 2       11  6019
securitytrails        1        4  4066
unk_bruteforcer    

  0%|          | 0/31 [00:00<?, ?it/s]

Corpus do dia 1 salvo com sucesso.
Corpus do dia 2 salvo com sucesso.
Corpus do dia 3 salvo com sucesso.
Corpus do dia 4 salvo com sucesso.
Corpus do dia 5 salvo com sucesso.
Corpus do dia 6 salvo com sucesso.
Corpus do dia 7 salvo com sucesso.
Corpus do dia 8 salvo com sucesso.
Corpus do dia 9 salvo com sucesso.
Corpus do dia 10 salvo com sucesso.
Corpus do dia 11 salvo com sucesso.
Corpus do dia 12 salvo com sucesso.
Corpus do dia 13 salvo com sucesso.
Corpus do dia 14 salvo com sucesso.
Corpus do dia 15 salvo com sucesso.
Corpus do dia 16 salvo com sucesso.
Corpus do dia 17 salvo com sucesso.
Corpus do dia 18 salvo com sucesso.
Corpus do dia 19 salvo com sucesso.
Corpus do dia 20 salvo com sucesso.
Corpus do dia 21 salvo com sucesso.
Corpus do dia 22 salvo com sucesso.
Corpus do dia 23 salvo com sucesso.
Corpus do dia 24 salvo com sucesso.
Corpus do dia 25 salvo com sucesso.
Corpus do dia 26 salvo com sucesso.
Corpus do dia 27 salvo com sucesso.
Corpus do dia 28 salvo com sucesso.
C

## Geração de embeddings (NLP) ##

In [3]:
from src.models.nlp import iWord2Vec
from tqdm.notebook import tqdm
import pickle
import glob
import os
import pandas as pd

# Inicializa o modelo
word2vec = iWord2Vec(c=5, e=128, epochs=1, seed=15)

# Processa os arquivos do corpus
for file in tqdm(sorted(glob.glob('../data/corpus/corpus_*.pkl'))):
    try:
        # Obtém o dia a partir do nome do arquivo
        day = file.split('/')[-1].replace('corpus_', '').replace('.pkl', '')
        
        # Valida se o dia é um número entre 1 e 31
        try:
            day_int = int(day)
            if not (1 <= day_int <= 31):
                print(f"Ignorando {file}: dia {day_int} inválido.")
                continue
        except ValueError:
            print(f"Ignorando {file}: nome do arquivo não contém um dia válido.")
            continue
        
        # Carrega o corpus
        with open(file, 'rb') as f:
            corpus_data = pickle.load(f)
        
        # Verifica a estrutura esperada
        if 'ports' not in corpus_data or 'corpus' not in corpus_data:
            print(f"Estrutura inválida em {file}. Ignorando.")
            continue
        
        port_order = corpus_data['ports']
        corpus = corpus_data['corpus']
        
        # Verifica se o corpus está vazio
        if not corpus:
            print(f"Corpus vazio em {file}. Ignorando.")
            continue
        
        # Treina ou atualiza o modelo conforme o dia
        if day_int == 1:
            print(f"Treinando modelo para o dia {day_int}...")
            word2vec.train(corpus)
        else:
            print(f"Atualizando modelo para o dia {day_int}...")
            word2vec.update(corpus)
        
        # Obtém os embeddings
        embeddings = word2vec.get_embeddings()
        if isinstance(embeddings, pd.DataFrame):
            embeddings.index.name = "src_ip"
            embeddings = embeddings.reset_index()
        else:
            embeddings = pd.DataFrame(embeddings)
            embeddings.insert(0, "src_ip", port_order)
        
        # Salva os embeddings em CSV
        os.makedirs("../data/nlp_embeddings", exist_ok=True)
        embeddings.to_csv(f'../data/nlp_embeddings/embeddings_idarkvec_{day}.csv', index=False)
        
        print(f"Embeddings do dia {day} salvos em '../data/nlp_embeddings/embeddings_idarkvec_{day}.csv'")
    
    except Exception as e:
        print(f"Erro ao processar {file}: {e}")


  0%|          | 0/31 [00:00<?, ?it/s]

Treinando modelo para o dia 1...
Embeddings do dia 1 salvos em '../data/nlp_embeddings/embeddings_idarkvec_1.csv'
Atualizando modelo para o dia 10...
Embeddings do dia 10 salvos em '../data/nlp_embeddings/embeddings_idarkvec_10.csv'
Atualizando modelo para o dia 11...
Embeddings do dia 11 salvos em '../data/nlp_embeddings/embeddings_idarkvec_11.csv'
Atualizando modelo para o dia 12...
Embeddings do dia 12 salvos em '../data/nlp_embeddings/embeddings_idarkvec_12.csv'
Atualizando modelo para o dia 13...
Embeddings do dia 13 salvos em '../data/nlp_embeddings/embeddings_idarkvec_13.csv'
Atualizando modelo para o dia 14...
Embeddings do dia 14 salvos em '../data/nlp_embeddings/embeddings_idarkvec_14.csv'
Atualizando modelo para o dia 15...
Embeddings do dia 15 salvos em '../data/nlp_embeddings/embeddings_idarkvec_15.csv'
Atualizando modelo para o dia 16...
Embeddings do dia 16 salvos em '../data/nlp_embeddings/embeddings_idarkvec_16.csv'
Atualizando modelo para o dia 17...
Embeddings do dia

## Classificação (NLP) ##

In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
import os

# Carrega o arquivo de ground truth contendo os rótulos reais
gt_file = 'ground_truth_simulado.csv'
if not os.path.exists(gt_file):
    raise FileNotFoundError(f"Arquivo de ground truth {gt_file} não encontrado.")

gt = pd.read_csv(gt_file)

gt['src_ip'] = gt['src_ip'].astype(str)  # Garante que os endereços IP sejam tratados como strings

if 'src_ip' not in gt.columns or 'label' not in gt.columns:
    raise ValueError("O ground truth deve conter as colunas 'src_ip' e 'label'.")

# Implementação de um classificador k-NN personalizado com vizinhos adaptativos
class KnnClassifier:
    def __init__(self, n_neighbors=3, metric='cosine'):
        self.n_neighbors = n_neighbors
        self.metric = metric
        self.model = None  # O modelo será inicializado na chamada do método `fit`

    def fit(self, X_train, y_train, scale_data=False):
        num_samples = len(y_train)

        k = min(self.n_neighbors, num_samples)  # Ajusta `k` para não ser maior que o número de amostras disponíveis
        if k < 1:
            raise ValueError("Número insuficiente de amostras para classificação k-NN.")

        self.model = KNeighborsClassifier(n_neighbors=k, metric=self.metric)

        if scale_data:
            self.scaler = StandardScaler()
            X_train = self.scaler.fit_transform(X_train)  # Normaliza os dados antes do treinamento

        self.model.fit(X_train, y_train)  # Treina o modelo com os dados fornecidos

    def predict(self, X_test, scale_data=False):
        if self.model is None:
            raise ValueError("O modelo ainda não foi treinado.")

        if scale_data:
            X_test = self.scaler.transform(X_test)  # Normaliza os dados antes da predição

        return self.model.predict(X_test)  # Retorna as previsões do modelo

# Pipeline de classificação
def classification_pipeline(embeddings, gt):
    print("\nVerificando estrutura dos embeddings...")
    print(embeddings.head())  # Exibe amostras dos embeddings para verificação

    if 'src_ip' not in embeddings.columns:
        raise ValueError("Arquivo de embeddings não contém 'src_ip'. Verifique o formato.")

    embeddings['src_ip'] = embeddings['src_ip'].astype(str)  # Garante que os endereços IP sejam strings

    # Mescla os embeddings com o ground truth com base no campo 'src_ip'
    embeddings = embeddings.merge(gt, on='src_ip', how='left').fillna('unknown')

    print("\nDistribuição de rótulos antes da filtragem:")
    print(embeddings['label'].value_counts())  # Mostra a distribuição dos rótulos antes da filtragem

    valid_labels = embeddings['label'].value_counts().index
    embeddings.loc[~embeddings['label'].isin(valid_labels), 'label'] = 'unknown'

    # Remove registros com rótulo 'unknown' apenas se houver outras classes válidas
    if (embeddings['label'] == 'unknown').sum() != len(embeddings):
        embeddings = embeddings[embeddings['label'] != 'unknown']

    print("\nDistribuição de rótulos após a filtragem:")
    print(embeddings['label'].value_counts())  # Exibe a distribuição dos rótulos após a filtragem

    # Verifica se há classes suficientes para treinar o modelo
    if len(np.unique(embeddings['label'].values)) < 2:
        raise ValueError("\nErro: quantidade insuficiente de classes válidas para classificação. Verifique a distribuição de rótulos.")

    X_train = embeddings.drop(columns=['label', 'src_ip'], errors='ignore').values  # Remove colunas não numéricas
    y_train = np.ravel(embeddings[['label']].values)  # Obtém os rótulos para treinamento

    knn = KnnClassifier(n_neighbors=3, metric='cosine')
    knn.fit(X_train, y_train, scale_data=True)  # Treina o classificador

    valid_indices = np.arange(len(y_train)).astype(int).flatten()

    y_true = y_train[valid_indices]  # Obtém os rótulos verdadeiros
    y_pred = knn.predict(X_train[valid_indices], scale_data=True)  # Obtém as previsões do modelo

    crep = classification_report(y_true, y_pred, labels=np.unique(y_true), output_dict=True)  # Gera o relatório de classificação
    return crep

# Carrega o arquivo de embeddings e executa a classificação
embedding_file = '../data/nlp_embeddings/embeddings_idarkvec_2.csv'
if not os.path.exists(embedding_file):
    raise FileNotFoundError(f"Arquivo de embeddings {embedding_file} não encontrado.")

embeddings = pd.read_csv(embedding_file)

print("\nVerificando conteúdo do arquivo:")
print(embeddings.head())  # Exibe amostras do arquivo de embeddings para depuração

if 'src_ip' not in embeddings.columns:
    raise KeyError("A coluna 'src_ip' está ausente no arquivo de embeddings. Verifique o formato dos dados.")

embeddings['src_ip'] = embeddings['src_ip'].astype(str)  # Converte os IPs para string para garantir consistência

classification_report = classification_pipeline(embeddings, gt)  # Executa o pipeline de classificação

print("\nRelatório de Classificação:")
print(pd.DataFrame(classification_report).transpose())  # Exibe o relatório de classificação final



Verificando conteúdo do arquivo:
         src_ip         0         1         2         3         4         5  \
0  192.168.1.26  0.006738  0.003012  0.003212  0.004935 -0.004036 -0.002431   
1  192.168.1.33  0.002965 -0.003778  0.000793 -0.001849 -0.000688 -0.006697   
2  192.168.1.40 -0.001823  0.007714  0.007047 -0.003224  0.007148  0.004393   
3  192.168.1.38 -0.005350  0.004542 -0.001045  0.005886 -0.000692 -0.002025   
4   192.168.1.9 -0.002121 -0.001018 -0.006599 -0.000633 -0.002287  0.005048   

          6         7         8  ...       118       119       120       121  \
0 -0.004646 -0.007112 -0.000888  ... -0.006956  0.003519  0.003514  0.004902   
1 -0.006737  0.003017 -0.004178  ... -0.004152  0.002504 -0.000645  0.005921   
2 -0.002487 -0.000087  0.002947  ...  0.004844  0.000537  0.001595  0.000475   
3  0.004510  0.002709 -0.003699  ... -0.000646 -0.004903 -0.001364 -0.001819   
4  0.000739 -0.003050 -0.006083  ... -0.003366 -0.004525 -0.001277 -0.005287   

        12